[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lorenzo-arcioni/BLAST-OUT-postprocessing/blob/main/AnnoReport.ipynb)

# Annotation Report: Multi-database Annotation Summary

This Jupyter notebook is designed to generate a report file based on annotation results obtained from annotation tools such as BLAST or Diamond. To use this notebook, users must have both the annotation results in TSV format and a table containing additional information about the transcripts. By leveraging the power of Jupyter notebooks, users can interactively visualize and analyze their annotation results, generating a comprehensive report that includes detailed statistics and data visualizations. The notebook's intuitive user interface and modular design make it easy to customize the report based on specific research needs. With its ability to quickly and efficiently generate reports from annotation data, this Jupyter notebook is an invaluable tool for researchers working with transcriptomic data.

If you need to transfer your data from a remote host please use this terminal.

In [ ]:
%pip install colab-xterm
%load_ext colabxterm
%xterm

In [1]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
pd.set_option('display.max_columns', None)

def get_transcripts_from_id(transcripts, table):
    transcripts = transcripts.unique()

    dic = dict()

    for t in transcripts:
        for x in table.transcript:
            if re.match(x, t):
                dic[t] = x
    return dic

def make_hyperlink(sseqid, database):
    
    if database.lower() == 'nr':
        protein_accession = sseqid.split(" ")[0]
        url = "https://www.ncbi.nlm.nih.gov/gene/?term={}"
    else:
        protein_accession = sseqid.split("|")[1]
        url = "https://www.uniprot.org/uniprotkb/{}/entry"
        
    return '=HYPERLINK("%s", "%s")' % (url.format(protein_accession), protein_accession)

def get_accession(sseqid, database):

    if database.lower() == 'nr':

        return sseqid.split(" ")[0]
    else:
        
        return sseqid.split("|")[1]

In this section, the user must customize the generation parameters (by appropriately modifying the variables) following the instructions in the comments in the cell below.

In [2]:
# Insert the names (or paths) of the tsv files
files = [
    "blastp/blastp_nr.tsv",
    "blastp/blastp_tr.tsv",
    "blastp/blastp_sp.tsv"
] 

# Insert the titles of the graph
title = "Blastp"

# Insert the databases names (the order must match the result files order)
databases_names =[
    "Nr", 
    "TrEMBL",
    "Swiss-Prot",
]

# Insert the table (with additional informations) path
table_path = "./blastp/tables/"

# Insert the path of the report
path = "./" + title + "_" + table_path.split("/")[-1]

# Set the outformat
# e.g. 
# outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar staxids sscinames sskingdoms skingdoms sphylums stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"
# If there are column names in the file then set outfmt = None
#outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"
outfmt = None
# Columns names (modify this list by inserting the column names of the report)
features = ["transcript", "row", "log2FoldChange", "padj", 
            "protein_accession", "sequence_identity", "alignment_length", 
            "evalue", "database", "gene", "locus_name", "sequence_description",
            "sequence_length", "organism", "protein_product"]


In [14]:
df = pd.DataFrame()
table = pd.read_csv(table_path, sep='\t')
table.index.name = 'transcript'

table.reset_index(inplace=True)

for i in range(len(files)):

    #Import the dataset
    if outfmt == None:
        df_tmp = pd.read_csv(files[i], sep="\t")
    else:
        df_tmp = pd.read_csv(files[i], sep="\t", names=outfmt.split())

    df_tmp['qseqid'] = df_tmp['qseqid'].map(get_transcripts_from_id(df_tmp['qseqid'], table))
    df_tmp['database'] = databases_names[i]
    df_tmp['row'] = title
    # df_tmp['sequence_identity'] = df_tmp.pident
    # df_tmp['alignment_length'] = df_tmp.length
    # df_tmp['evalue'] = df_tmp.evalue
    # df_tmp['sequence_description'] = df_tmp.stitle
    # df_tmp['sequence_length'] = df_tmp.slen

    df_tmp.rename(columns={'pident': 'sequence_identity',
                           'length': 'alignment_length',
                           'stitle': 'sequence_description',
                           'slen': 'sequence_length'
    }, inplace=True)

    if "OS=" not in df_tmp.sequence_description[0]:    
        def get_sciname(x):
            
            try:
                os_index = - x[::-1].index('[')
            except:
                return ""

            return x[os_index:-1]

        # Useful functions
        def get_protein_function(x):

            x_l = x.split(" ")

            try:
                nex = ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith('[')))])
            except:
                return ""

            return nex
        
        def get_locus_name(x):
            return None
        
        def get_gene(x):
            return None
    else:
        def get_sciname(x):

            os_index = x.index('OS=')
            ox_index = x.index('OX=')

            return x[os_index+3:ox_index-1]

        # Useful functions
        def get_protein_function(x):

            x_l = x.split(" ")

            return ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith('OS=')))])
        
        def get_locus_name(x):
            return x.split("|")[2]
        
        def get_gene(x):

            try:
                gn_index = x.index('GN=')
                pe_index = x.index('PE=')
            except:
                return None
            return x[gn_index+3:pe_index-1]
        
    df_tmp['gene'] = df_tmp.sequence_description.apply(lambda x: get_gene(x))
    df_tmp['organism'] = df_tmp.sequence_description.apply(lambda x: get_sciname(x))
    df_tmp['protein_accession'] = df_tmp.apply(lambda x: get_accession(x.sseqid, x.database), axis=1)
    df_tmp['protein_product'] = df_tmp.sequence_description.apply(lambda x: get_protein_function(x))
    df_tmp['locus_name'] = df_tmp.sseqid.apply(lambda x: get_locus_name(x))

    df_tmp = pd.merge(df_tmp, table, left_on='qseqid', right_on='transcript', how='left')

    df = pd.concat([df, df_tmp[features]])

df.sort_values(['transcript', 'evalue'], inplace=True)

df.reset_index(drop=True, inplace=True)

In [15]:
df.head()

,transcript,row,log2FoldChange,padj,protein_accession,sequence_identity,alignment_length,evalue,database,gene,locus_name,sequence_description,sequence_length,organism,protein_product
0,GKNQ01001237.1,Blastp,-1.121591,0.018273,VDM39180.1,81.1,928,0.0,Nr,None,None,VDM39180.1 unnamed protein product [Toxocara c...,936,Toxocara canis,unnamed protein product
1,GKNQ01001237.1,Blastp,-1.121591,0.018273,TMS38063.1,72.6,916,0.0,Nr,None,None,TMS38063.1 hypothetical protein L596_004871 [S...,939,Steinernema carpocapsae,hypothetical protein L596_004871
2,GKNQ01001237.1,Blastp,-1.121591,0.018273,CDJ94054.1,70.9,929,0.0,Nr,None,None,CDJ94054.1 Alanine dehydrogenase PNT and Sacch...,931,Haemonchus contortus,Alanine dehydrogenase PNT and Saccharopine deh...
3,GKNQ01001237.1,Blastp,-1.121591,0.018273,VDO37093.1,69.3,948,0.0,Nr,None,None,VDO37093.1 unnamed protein product [Haemonchus...,951,Haemonchus placei,unnamed protein product
4,GKNQ01001237.1,Blastp,-1.121591,0.018273,EYC41722.1,70.5,914,0.0,Nr,None,None,EYC41722.1 hypothetical protein Y032_0558g3408...,932,Ancylostoma ceylanicum,hypothetical protein Y032_0558g3408


If you want to remove repetitive information (e.g. trasncript name ecc..) run the cell below (you can also modify given list).

In [16]:
df.loc[df.duplicated(subset=['transcript', 'row', 'log2FoldChange', 'padj']), 'transcript':'padj'] = ''
unmatched_transcript = pd.Series(list(set(table.transcript) - set(df.transcript)), 
                                 name='unmatched_transcript')

Run the cell below to save the report in tsv format.

In [17]:
df.to_csv(path + '.tsv', sep='\t')

Run the cell below to save the report in xlsx format.

In [18]:
from openpyxl.utils import get_column_letter

df_writer = pd.ExcelWriter(path + '.xlsx') 

# Scrivi il dataframe nel foglio di lavoro
df.to_excel(df_writer, sheet_name='report', index=False)

# Ottieni l'oggetto Worksheet dall'oggetto ExcelWriter
worksheet = df_writer.sheets['report']

# Imposta la larghezza delle colonne in modo automatico
for column in range(df.shape[1]):
    column_length = max(df.iloc[:, column].astype(str).map(len).max(), len(df.columns[column])) + 3
    column_letter = get_column_letter(column + 1)
    worksheet.column_dimensions[column_letter].width = column_length

# Salva il foglio di lavoro
df_writer.close()

# Calcolo i trascritti che non hano mappato
df_writer = pd.ExcelWriter(path + "_unmatch" + ".xlsx")
unmatched_transcript.to_excel(df_writer, sheet_name='unmatched transcripts', index=False)

# Ottieni l'oggetto Worksheet dall'oggetto ExcelWriter
worksheet = df_writer.sheets['unmatched transcripts']

# Imposta la larghezza delle colonne in modo automatico
worksheet.column_dimensions['A'].width = max(unmatched_transcript.astype(str).map(len).max(), len(unmatched_transcript.name)) + 3

# Salva il foglio di lavoro
df_writer.close()